In [1]:
from shapely.geometry import Point, Polygon
from shapely import wkt
import skmob
from skmob.preprocessing import filtering, detection
from skmob.utils.plot import plot_gdf
from skmob.tessellation import tilers
import pandas as pd
import geopandas as gpd
import webbrowser
import numpy as np
import networkx as nx

pd.set_option('display.max_columns', 500)

In [2]:
meters = 500

# Utilities #

In [3]:
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None


In [4]:
def to_Adj_Matrix(df):
    df_np = df.to_numpy().astype(int)
    n = max(max(origin, destination) for origin, destination, flow in df_np )
    
    matrix = np.zeros((n+1, n+1))
    for origin, destination, flow in df_np:
        matrix[origin][destination] = flow 
    return matrix
    

In [5]:
def pad_with_zeros(A, r, c):
   out = np.zeros((r, c))
   r_, c_ = np.shape(A)
   out[0:r_, 0:c_] = A
   return out

# NYC #

In [6]:
tess_nyc = tilers.tiler.get("squared", meters=meters, base_shape="New York City, New York")

/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
tess_nyc.head(10)

,tile_ID,geometry
0,0,"POLYGON ((-74.25909 40.48765, -74.25909 40.491..."
1,1,"POLYGON ((-74.25909 40.49106, -74.25909 40.494..."
2,2,"POLYGON ((-74.25909 40.49448, -74.25909 40.497..."
3,3,"POLYGON ((-74.25909 40.49790, -74.25909 40.501..."
4,4,"POLYGON ((-74.25909 40.50131, -74.25909 40.504..."
5,5,"POLYGON ((-74.25909 40.50473, -74.25909 40.508..."
6,6,"POLYGON ((-74.25909 40.50814, -74.25909 40.511..."
7,7,"POLYGON ((-74.25909 40.51156, -74.25909 40.514..."
8,8,"POLYGON ((-74.25909 40.51497, -74.25909 40.518..."
9,9,"POLYGON ((-74.25460 40.48423, -74.25460 40.487..."


In [8]:
#plot_gdf(tess_nyc, popup_features=['tile_ID'])

In [9]:
df_nyc = pd.read_csv('data/201802-citibike-tripdata.csv.zip')
#df_nyc = df_nyc.drop_duplicates()

df_nyc['id'] = df_nyc.apply(lambda x: hash(tuple(x)), axis = 1)

df_nyc.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,id
0,771,2018-02-01 00:14:16.4120,2018-02-01 00:27:08.2290,72,W 52 St & 11 Ave,40.767272,-73.993929,379,W 31 St & 7 Ave,40.749156,-73.991600,14536,Subscriber,1952,1,5959634247907276963
1,264,2018-02-01 05:14:45.1790,2018-02-01 05:19:09.6860,72,W 52 St & 11 Ave,40.767272,-73.993929,478,11 Ave & W 41 St,40.760301,-73.998842,32820,Subscriber,1965,1,-2125787656579734331
2,819,2018-02-01 06:48:55.2290,2018-02-01 07:02:35.0290,72,W 52 St & 11 Ave,40.767272,-73.993929,405,Washington St & Gansevoort St,40.739323,-74.008119,16131,Subscriber,1968,1,2914677924339393290
3,646,2018-02-01 07:12:50.1740,2018-02-01 07:23:36.5280,72,W 52 St & 11 Ave,40.767272,-73.993929,2006,Central Park S & 6 Ave,40.765909,-73.976342,20831,Subscriber,1990,2,8590760973250600986
4,1312,2018-02-01 07:46:48.8750,2018-02-01 08:08:41.5430,72,W 52 St & 11 Ave,40.767272,-73.993929,435,W 21 St & 6 Ave,40.741740,-73.994156,15899,Subscriber,1957,1,-2542080023085159108


In [10]:
df_nyc.size

13489824

In [11]:
df_nyc['geometry_source'] = [Point(xy) for xy in zip(df_nyc['start station longitude'],df_nyc['start station latitude'])]
df_nyc['geometry_target'] = [Point(xy) for xy in zip(df_nyc['end station longitude'],df_nyc['end station latitude'])]



columns = ['tripduration', 'start station name', 'start station latitude', 'start station longitude', 'end station name', 'end station latitude', 'end station longitude', 'usertype', 'birth year', 'gender', 'starttime', 'stoptime', 'bikeid']
df_nyc = df_nyc.drop(columns, axis = 1) 

df_nyc.head()



,start station id,end station id,id,geometry_source,geometry_target
0,72,379,5959634247907276963,POINT (-73.99392888 40.76727216),POINT (-73.99160000000001 40.749156)
1,72,478,-2125787656579734331,POINT (-73.99392888 40.76727216),POINT (-73.99884222 40.76030096)
2,72,405,2914677924339393290,POINT (-73.99392888 40.76727216),POINT (-74.00811899999999 40.739323)
3,72,2006,8590760973250600986,POINT (-73.99392888 40.76727216),POINT (-73.97634151 40.76590936)
4,72,435,-2542080023085159108,POINT (-73.99392888 40.76727216),POINT (-73.99415556 40.74173969)


## Source ##

In [12]:
gdf_1 = gpd.GeoDataFrame(df_nyc, crs="EPSG:4326", geometry=df_nyc['geometry_source']).drop(['geometry_target', 'geometry_source'],axis =1)

gdf_1.head(10)

,start station id,end station id,id,geometry
0,72,379,5959634247907276963,POINT (-73.99393 40.76727)
1,72,478,-2125787656579734331,POINT (-73.99393 40.76727)
2,72,405,2914677924339393290,POINT (-73.99393 40.76727)
3,72,2006,8590760973250600986,POINT (-73.99393 40.76727)
4,72,435,-2542080023085159108,POINT (-73.99393 40.76727)
5,72,173,7320605356003391474,POINT (-73.99393 40.76727)
6,72,514,2336758560678789615,POINT (-73.99393 40.76727)
7,72,426,3763169120031323833,POINT (-73.99393 40.76727)
8,72,3173,-4713880486831951968,POINT (-73.99393 40.76727)
9,72,458,-1976794815170581541,POINT (-73.99393 40.76727)


In [13]:
res_merge_source = gpd.sjoin(tess_nyc, gdf_1, how='right', op='contains')
res_merge_source.dropna(inplace=True)
res_merge_source.reset_index(drop=True, inplace=True)


/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  """Entry point for launching an IPython kernel.


In [14]:
res_merge_source.head(20000)

,index_left,tile_ID,start station id,end station id,id,geometry
0,2740.0,2740,72,379,5959634247907276963,POINT (-73.99393 40.76727)
1,2740.0,2740,72,478,-2125787656579734331,POINT (-73.99393 40.76727)
2,2740.0,2740,72,405,2914677924339393290,POINT (-73.99393 40.76727)
3,2740.0,2740,72,2006,8590760973250600986,POINT (-73.99393 40.76727)
4,2740.0,2740,72,435,-2542080023085159108,POINT (-73.99393 40.76727)
...,...,...,...,...,...,...
19995,2648.0,2648,161,253,1771380200743328173,POINT (-73.99810 40.72917)
19996,2648.0,2648,161,496,1842606181116068097,POINT (-73.99810 40.72917)
19997,2648.0,2648,161,265,-4691179470513562178,POINT (-73.99810 40.72917)
19998,2648.0,2648,161,265,4197281299082615498,POINT (-73.99810 40.72917)


## Dest ##

In [15]:
gdf_2 = gpd.GeoDataFrame(df_nyc, crs="EPSG:4326", geometry=df_nyc['geometry_target']).drop(['geometry_target', 'geometry_source'],axis =1)

gdf_1.head(10)


,start station id,end station id,id,geometry
0,72,379,5959634247907276963,POINT (-73.99393 40.76727)
1,72,478,-2125787656579734331,POINT (-73.99393 40.76727)
2,72,405,2914677924339393290,POINT (-73.99393 40.76727)
3,72,2006,8590760973250600986,POINT (-73.99393 40.76727)
4,72,435,-2542080023085159108,POINT (-73.99393 40.76727)
5,72,173,7320605356003391474,POINT (-73.99393 40.76727)
6,72,514,2336758560678789615,POINT (-73.99393 40.76727)
7,72,426,3763169120031323833,POINT (-73.99393 40.76727)
8,72,3173,-4713880486831951968,POINT (-73.99393 40.76727)
9,72,458,-1976794815170581541,POINT (-73.99393 40.76727)


In [16]:
res_merge_dest = gpd.sjoin(tess_nyc, gdf_2, how='right', op='contains')
res_merge_dest.dropna(inplace=True)
res_merge_dest.reset_index(drop=True,inplace=True)

/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  """Entry point for launching an IPython kernel.


In [17]:
res_merge_dest.head(15)

,index_left,tile_ID,start station id,end station id,id,geometry
0,2734.0,2734,72,379,5959634247907276963,POINT (-73.99160 40.74916)
1,2579.0,2579,72,478,-2125787656579734331,POINT (-73.99884 40.76030)
2,2423.0,2423,72,405,2914677924339393290,POINT (-74.00812 40.73932)
3,2985.0,2985,72,2006,8590760973250600986,POINT (-73.97634 40.76591)
4,2652.0,2652,72,435,-2542080023085159108,POINT (-73.99416 40.74174)
5,2900.0,2900,72,173,7320605356003391474,POINT (-73.98453 40.76068)
6,2580.0,2580,72,514,2336758560678789615,POINT (-74.00278 40.76088)
7,2344.0,2344,72,426,3763169120031323833,POINT (-74.01322 40.71755)
8,2823.0,2823,72,3173,-4713880486831951968,POINT (-73.98889 40.77751)
9,2501.0,2501,72,458,-1976794815170581541,POINT (-74.00523 40.75140)


## Merge ##

In [18]:
fdf = res_merge_dest.merge(res_merge_source, how='inner', on=['id'])
fdf = fdf[['tile_ID_x','tile_ID_y']]


In [19]:
fdf.head(10)

,tile_ID_x,tile_ID_y
0,2734,2740
1,2579,2740
2,2423,2740
3,2985,2740
4,2652,2740
5,2900,2740
6,2580,2740
7,2344,2740
8,2823,2740
9,2501,2740


In [20]:
flusso = fdf.groupby(["tile_ID_x", "tile_ID_y"]).size().reset_index(name="flow")
flusso = flusso.rename(columns={'tile_ID_x': 'origin', 'tile_ID_y':'destination'})

In [21]:
fdf_nyc = skmob.FlowDataFrame(data = flusso,tessellation=tess_nyc, tile_id='tile_ID', origin ='origin', 
                              destination = 'destination', flow = 'flow')

In [22]:
fdf_nyc.head()

,origin,destination,flow
0,2254,2254,40
1,2271,2271,55
2,2271,2272,44
3,2271,2274,93
4,2271,2339,51


In [23]:
fdf_nyc.to_csv("fdf_nyc.csv")

In [24]:
fdf_nyc_np = to_Adj_Matrix(fdf_nyc)

fdf_nyc_np

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 9.]])

In [48]:
fdf_nyc_np[2271,2272]

44.0

# ____________________________________________________________ #

# ____________________________________________________________ #

# PORTO #

In [26]:
tess_por = tilers.tiler.get("squared", meters=meters, base_shape="Porto, Área Metropolitana do Porto, North, Portugal")

/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [27]:
df_por = pd.read_csv('data/porto_trajectories_all.csv')
df_por = df_por.drop_duplicates()


mask = df_por.trajectory_id.duplicated(keep=False)
prova = df_por[mask]
prova = df_por.drop_duplicates(subset=['trajectory_id'])
df_por = prova



df_por['geometry_source'] = df_por.source_point.apply(wkt_loads)
df_por['geometry_target'] = df_por.target_point.apply(wkt_loads)


In [28]:
df_por.head()

,taxi_id,trajectory_id,timestamp,source_point,target_point,geometry_source,geometry_target
0,20000589,1372636858620000589,2013-07-01 00:00:58,POINT(-8.618643 41.141412),POINT(-8.630838 41.154489),POINT (-8.618643 41.141412),POINT (-8.630838000000001 41.154489)
1,20000596,1372637303620000596,2013-07-01 00:08:23,POINT(-8.639847 41.159826),POINT(-8.66574 41.170671),POINT (-8.639847 41.159826),POINT (-8.66574 41.170671)
2,20000320,1372636951620000320,2013-07-01 00:02:31,POINT(-8.612964 41.140359),POINT(-8.61597 41.14053),POINT (-8.612964 41.140359),POINT (-8.615970000000001 41.14053)
3,20000520,1372636854620000520,2013-07-01 00:00:54,POINT(-8.574678 41.151951),POINT(-8.607996 41.142915),POINT (-8.574678 41.151951),POINT (-8.607996 41.142915)
4,20000337,1372637091620000337,2013-07-01 00:04:51,POINT(-8.645994 41.18049),POINT(-8.687268 41.178087),POINT (-8.645994 41.18049),POINT (-8.687268 41.178087)


In [29]:
df_por.size

11719043

## Source ##

In [30]:
gdf_1 = gpd.GeoDataFrame(df_por, crs="EPSG:4326", geometry=df_por['geometry_source'])

gdf_1.head()


,taxi_id,trajectory_id,timestamp,source_point,target_point,geometry_source,geometry_target,geometry
0,20000589,1372636858620000589,2013-07-01 00:00:58,POINT(-8.618643 41.141412),POINT(-8.630838 41.154489),POINT (-8.618643 41.141412),POINT (-8.630838000000001 41.154489),POINT (-8.61864 41.14141)
1,20000596,1372637303620000596,2013-07-01 00:08:23,POINT(-8.639847 41.159826),POINT(-8.66574 41.170671),POINT (-8.639847 41.159826),POINT (-8.66574 41.170671),POINT (-8.63985 41.15983)
2,20000320,1372636951620000320,2013-07-01 00:02:31,POINT(-8.612964 41.140359),POINT(-8.61597 41.14053),POINT (-8.612964 41.140359),POINT (-8.615970000000001 41.14053),POINT (-8.61296 41.14036)
3,20000520,1372636854620000520,2013-07-01 00:00:54,POINT(-8.574678 41.151951),POINT(-8.607996 41.142915),POINT (-8.574678 41.151951),POINT (-8.607996 41.142915),POINT (-8.57468 41.15195)
4,20000337,1372637091620000337,2013-07-01 00:04:51,POINT(-8.645994 41.18049),POINT(-8.687268 41.178087),POINT (-8.645994 41.18049),POINT (-8.687268 41.178087),POINT (-8.64599 41.18049)


In [31]:
res_merge_source = gpd.sjoin(tess_por, gdf_1, how='right', op='contains')
res_merge_source.dropna(inplace=True)
res_merge_source.reset_index(drop=True, inplace=True)


/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  """Entry point for launching an IPython kernel.


In [32]:
res_merge_source.head()

,index_left,tile_ID,taxi_id,trajectory_id,timestamp,source_point,target_point,geometry_source,geometry_target,geometry
0,170.0,170,20000589,1372636858620000589,2013-07-01 00:00:58,POINT(-8.618643 41.141412),POINT(-8.630838 41.154489),POINT (-8.618643 41.141412),POINT (-8.630838000000001 41.154489),POINT (-8.61864 41.14141)
1,114.0,114,20000596,1372637303620000596,2013-07-01 00:08:23,POINT(-8.639847 41.159826),POINT(-8.66574 41.170671),POINT (-8.639847 41.159826),POINT (-8.66574 41.170671),POINT (-8.63985 41.15983)
2,184.0,184,20000320,1372636951620000320,2013-07-01 00:02:31,POINT(-8.612964 41.140359),POINT(-8.61597 41.14053),POINT (-8.612964 41.140359),POINT (-8.615970000000001 41.14053),POINT (-8.61296 41.14036)
3,298.0,298,20000520,1372636854620000520,2013-07-01 00:00:54,POINT(-8.574678 41.151951),POINT(-8.607996 41.142915),POINT (-8.574678 41.151951),POINT (-8.607996 41.142915),POINT (-8.57468 41.15195)
4,108.0,108,20000337,1372637091620000337,2013-07-01 00:04:51,POINT(-8.645994 41.18049),POINT(-8.687268 41.178087),POINT (-8.645994 41.18049),POINT (-8.687268 41.178087),POINT (-8.64599 41.18049)


## Dest ##

In [33]:
gdf_2 = gpd.GeoDataFrame(df_por, crs="EPSG:4326", geometry=df_por['geometry_target'])

In [34]:
res_merge_dest = gpd.sjoin(tess_por, gdf_2, how='right', op='contains')
res_merge_dest.dropna(inplace=True)
res_merge_dest.reset_index(drop=True,inplace=True)

/home/giovanni/anaconda3/envs/skmob/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  """Entry point for launching an IPython kernel.


In [35]:
res_merge_dest.head()

,index_left,tile_ID,taxi_id,trajectory_id,timestamp,source_point,target_point,geometry_source,geometry_target,geometry
0,136.0,136,20000589,1372636858620000589,2013-07-01 00:00:58,POINT(-8.618643 41.141412),POINT(-8.630838 41.154489),POINT (-8.618643 41.141412),POINT (-8.630838000000001 41.154489),POINT (-8.63084 41.15449)
1,47.0,47,20000596,1372637303620000596,2013-07-01 00:08:23,POINT(-8.639847 41.159826),POINT(-8.66574 41.170671),POINT (-8.639847 41.159826),POINT (-8.66574 41.170671),POINT (-8.66574 41.17067)
2,170.0,170,20000320,1372636951620000320,2013-07-01 00:02:31,POINT(-8.612964 41.140359),POINT(-8.61597 41.14053),POINT (-8.612964 41.140359),POINT (-8.615970000000001 41.14053),POINT (-8.61597 41.14053)
3,199.0,199,20000520,1372636854620000520,2013-07-01 00:00:54,POINT(-8.574678 41.151951),POINT(-8.607996 41.142915),POINT (-8.574678 41.151951),POINT (-8.607996 41.142915),POINT (-8.60800 41.14292)
4,300.0,300,20000231,1372636965620000231,2013-07-01 00:02:45,POINT(-8.615502 41.140674),POINT(-8.578224 41.160717),POINT (-8.615501999999999 41.140674),POINT (-8.578224000000001 41.160717),POINT (-8.57822 41.16072)


## Merge ##

In [36]:
fdf = res_merge_dest.merge(res_merge_source, how='inner', on='trajectory_id')

In [37]:
fdf = fdf[['tile_ID_x','tile_ID_y']]

flusso = fdf.groupby(["tile_ID_x", "tile_ID_y"]).size().reset_index(name="flow")
flusso = flusso.rename(columns={'tile_ID_x': 'origin', 'tile_ID_y':'destination'})

In [38]:
flusso.head()

,origin,destination,flow
0,0,1,1
1,0,114,5
2,0,134,1
3,0,136,1
4,0,149,1


In [39]:
fdf_porto = skmob.FlowDataFrame(data = flusso,tessellation=tess_por, tile_id='tile_ID', origin ='origin', destination = 'destination', flow = 'flow')

In [40]:
fdf_porto.head()

,origin,destination,flow
0,0,1,1
1,0,114,5
2,0,134,1
3,0,136,1
4,0,149,1


In [41]:
fdf_porto.to_csv("flow-porto-raw.csv")

In [42]:
fdf_por_np = to_Adj_Matrix(fdf_porto)

fdf_por_np

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., 54.,  7., ...,  0.,  0.,  0.],
       [ 0.,  3.,  8., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [43]:
fdf_por_np[0,114]

5.0

# PADDING #

In [44]:
fdf_por_np.shape

(344, 344)

In [45]:
fdf_nyc_np.shape

(4702, 4702)

In [46]:
por_padded = pad_with_zeros(fdf_por_np, fdf_nyc_np.shape[0], fdf_nyc_np.shape[1])

In [47]:
por_padded.shape

(4702, 4702)